In [1]:
import keras
import math
import pandas as pd
import numpy as np
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
import time
from torch.utils.data import Dataset, DataLoader
import pickle
pd.set_option('display.max_rows', 500)
import os
import tensorflow as tf
import torch
import torch.nn as nn
from math import sqrt
# import rmse from sklearn
from sklearn.metrics import mean_squared_error


# define random seeds for Neural Networks
torch.manual_seed(0)
np.random.seed(0)
tf.random.set_seed(0)
# ignore warnings jupyter notebook
import warnings
warnings.filterwarnings('ignore')

# OWRI FRAMEWORK

In [2]:
with open('../data/METR-LA/METR_OWRI/featured_fpds_raw.pickle', 'rb') as f:
    results = pickle.load(f)

In [3]:
# load data of correlated results from pickle file
with open('../results/METR-LA/outlier_scores/AE/correlated_results.pickle', 'rb') as f:
    correlated_results = pickle.load(f)

In [4]:
# get target intersections for each trajectory and direction
target_intersections=["717445","773062","737529","769418", "761599", "717587", "717458"]

In [5]:
def merge_trejectory_data(results, trajectory, direction):
    data = pd.DataFrame()
    for intersection_name in results[target]['raw']:
        intersection = results[target]['raw'][intersection_name]
        intersection = intersection.rename(columns={"cars": intersection_name})
        intersection = intersection.set_index(pd.DatetimeIndex(intersection['timestamp']))
        intersection = intersection.drop(columns=['timestamp'])
        data = pd.merge(data, intersection, left_index=True, right_index=True, how='outer')
    data.dropna(inplace=True)
    return data

In [6]:
def merge_trejectory_data_metr(results):
    data = pd.DataFrame()
    for intersection_name in results['raw']:
        intersection = results['raw'][intersection_name]
        intersection = intersection.rename(columns={"cars": intersection_name})
        intersection = intersection.set_index(pd.DatetimeIndex(intersection['timestamp']))
        intersection = intersection.drop(columns=['timestamp'])
        data = pd.merge(data, intersection, left_index=True, right_index=True, how='outer')
    data.dropna(inplace=True)
    return data

In [7]:
def preprocess_df(df,n_obs, n_features, sequence_length):
    #do scaling:
    scaler = StandardScaler()
    df_train = df.values
    train_X, train_y = df_train[:, :n_obs], df_train[:, -n_features]
    scl = scaler.fit(train_X) # fit only on training data
    train_X = scl.transform(train_X)
    train_X = train_X.reshape((train_X.shape[0], sequence_length, n_features))
    return train_X, train_y, scl

In [8]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [9]:
class LSTM_uni(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, device = 'mps',layer_dim=1, dropout_prob = 0.2):
        super(LSTM_uni, self).__init__()
        self.hidden_dim = hidden_dim # number of hidden units in hidden state
        self.layer_dim = layer_dim # number of stacked lstm layers
        # batch_first=True causes input/output tensors to be of shape
        # (batch_dim, seq_dim, feature_dim)
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True, dropout=dropout_prob)
        self.fc = nn.Linear(hidden_dim, output_dim) # fully connected layer

    def forward(self, x, future=False):
        # input x is expected to be of shape (batch_dim, seq_dim, feature_dim)
        # hidden and cell states are expected along with input x in LSTMs = (h_0, c_0)
        # Initialize hidden state with zeros (layer_dim, batch_size, hidden_dim)
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim, device=device).requires_grad_()
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim, device=device).requires_grad_()

        # LSTM output is Outputs: output, (h_n, c_n)
        # output is of shape (batch_dim, seq_dim, hidden_dim), h_n and c_n are of shape (layer_dim, batch_dim, hidden_dim)
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = out[:, -1, :] # only take the last output of the sequence
        out = self.fc(out) # fully connected layer

        return out

In [10]:
def train_model(model, train_X,train_y, loss_fn, optimiser, device, epochs=100):
    history = {}
    history['train_loss'] = []

    train_X_loader = DataLoader(train_X, batch_size=128, shuffle=False)
    train_y_loader = DataLoader(train_y, batch_size=128, shuffle=False)

    for epoch in range(epochs):
        history[epoch] = []
        ep_start = time.time()
        running_loss = 0.0
        for bx, data in enumerate(zip(train_X_loader,train_y_loader)):
            X = data[0].to(device)
            y = data[1].to(device)
            bt = model(X)
            loss = loss_fn(bt.reshape(-1), y.reshape(-1)) # calculate loss for input and recreated output
            history[epoch].append(loss.item())
            optimiser.zero_grad()
            loss.backward()
            optimiser.step()
            running_loss += loss.item()
        epoch_loss = running_loss/train_X.shape[0]
        history['train_loss'].append(epoch_loss)

    return history, model
    

In [11]:
def model_evaluation( model, test_X, device):
    test_X_loader = DataLoader(test_X, batch_size=128, shuffle=False)
    model = model.eval()
    preds = []
    with torch.no_grad():
        for bx, data in enumerate(test_X_loader):
            X = data.to(device)
            bt = model(X)
            preds.append(bt.cpu().numpy())
    preds = np.vstack(preds)
    preds = preds.reshape(-1)
    return preds

In [12]:
device = 'mps' if torch.backends.mps.is_available() else 'cpu'

In [13]:
# errors={}
# dfs={}
# intersection_arrays = []
# for trajectory in results.keys():
#     errors[trajectory]={}
#     print("\n \n Starting trajectory: {}".format(trajectory))
#     for direction in results[trajectory]:
#         target = target_intersections[target]
#         errors[target]={}
#         print("Starting direction: {}".format(direction))
#         for threshold in thresholds:
#             errors[target][threshold]={}
#             print("Starting threshold: {}".format(threshold))
#             # ------------------------------------ data processing ---------------------------------------- #
#             data = merge_trejectory_data(results, trajectory, direction)# get raw data of the current trajectory and direction
#             ae_score = correlated_results[target] # AE scores of the current trajectory and direction
#             number_of_cols = math.ceil(len(ae_score.columns)*threshold) # number of outlier weighted intersections
#             if number_of_cols==0: # if threshold is 0, then use the target intersection only
#                 number_of_cols=1
#             top_corr_df = ae_score.corr()[target].sort_values(ascending=False)[:number_of_cols] # get the top correlated intersections
#             isct_inc = top_corr_df.index.tolist()
#             df = data[isct_inc].copy(deep=True)
#             df = df[ [target] + [ col for col in df.columns if col != target ] ]  #move target var to front of DF
#             df = df.mul(top_corr_df, axis=1)
#             df = df.astype('float32')
#             sequence_length = 12 # number of time steps to look back
#             n_features = len(isct_inc) # number of features (correlated intersections)
#             output_pred = 1 # number of time steps to predict
#             n_obs = sequence_length * n_features # number of columns in the input
#             reframed = series_to_supervised(df, sequence_length, output_pred)
#             train_X, train_y, test_X, test_y, scl = preprocess_df(reframed, n_obs, n_features, sequence_length)
#             device = 'mps' if torch.backends.mps.is_available() else 'cpu'

# #             # # ------------------------------------ modelling ---------------------------------------------- #
#             # define model, loss function and optimizer
#             model = LSTM_uni(input_dim = n_features, hidden_dim = 32, layer_dim = 1, output_dim = 1, dropout_prob= 0.2)
#             model = model.to(device)
#             loss_fn = torch.nn.MSELoss()
#             optimiser = torch.optim.Adam(model.parameters(), lr=0.01)
#             start = time.time()
#             history = train_model(model, train_X,train_y, loss_fn, optimiser, device, epochs=50)
#             end = time.time()
#             print("Training time: {}".format(end-start))


#             # ------------------------------------ evaluation ---------------------------------------------- #
#             yhat = model_evaluation( model, test_X , device)
#             errors[target][threshold]['RMSE'] = sqrt(mean_squared_error(yhat,test_y))
#             errors[target][threshold]['MAE'] = mean_absolute_error(yhat,test_y)
#             errors[target][threshold]['history'] = history
#             errors[target][threshold]['df'] = pd.DataFrame({"Real":test_y,"Predicted":yhat})
#             errors[target][threshold]['train_time'] = end-start


# # save errors in save path as pickle file
# with open(results_save_path, 'wb') as handle:
#     pickle.dump(errors, handle)

In [14]:
def load_ckp(checkpoint_fpath, model, optimizer):
    checkpoint = torch.load(checkpoint_fpath)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    return model, optimizer

In [15]:
# results save path
base_result_path = '../results/METR-LA/real_time_modeling/'
# exp_name = 'univariate_AE_incremental_update_results'
# results_save_path = os.path.join(base_result_path, exp_name)

In [36]:
def test_then_train_weighted_model(data, ae_score, number_of_cols, target, results_load_path, results_save_path, base_mode_path, pre_trained=True):
    result_dict = {}

    # ------------------------------------ data processing ---------------------------------------- #
    top_corr_df = ae_score.corr()[target].sort_values(ascending=False)[:number_of_cols] # get the top correlated intersections
    isct_inc = top_corr_df.index.tolist()
    print("correlated intersections: {}".format(isct_inc))
    df = data[isct_inc].copy(deep=True)
    df = df.mul(top_corr_df, axis=1)
    df = df.astype('float32')
    sequence_length = 12 # number of time steps to look back
    n_features = len(isct_inc) # number of features (correlated intersections)
    output_pred = 1 # number of time steps to predict
    n_obs = sequence_length * n_features # number of columns in the input
    reframed = series_to_supervised(df, sequence_length, output_pred)
    train_X, train_y, scl = preprocess_df(reframed, n_obs, n_features, sequence_length)


# # # ------------------------------------ modelling ---------------------------------------------- #
    # define model, loss function and optimizer
    model = LSTM_uni(input_dim = n_features, hidden_dim = 32, layer_dim = 1, output_dim = output_pred, dropout_prob= 0.2)
    model = model.to(device)
    loss_fn = torch.nn.MSELoss()
    optimiser = torch.optim.Adam(model.parameters(), lr=0.01)
    if pre_trained==True:
        model, optimiser = load_ckp(results_load_path, model, optimiser)
    

    # ------------------------------------ testing ---------------------------------------------- #
    yhat = model_evaluation( model, train_X , device)
    result_dict['RMSE'] = sqrt(mean_squared_error(yhat,train_y))
    result_dict['MAE'] = mean_absolute_error(yhat,train_y)
    result_dict['df'] = pd.DataFrame({"Real":train_y,"Predicted":yhat})
    print("RMSE: {}".format(result_dict['RMSE']))


   # ------------------------------------ training ---------------------------------------------- #
    start = time.time()
    if pre_trained==False:
        history, model = train_model(model, train_X,train_y, loss_fn, optimiser, device, epochs=50)
    else:
        history, model = train_model(model, train_X,train_y, loss_fn, optimiser, device, epochs=1)
    end = time.time()
    checkpoint = {'state_dict': model.state_dict(),'optimizer': optimiser.state_dict()}
    torch.save(checkpoint, results_save_path)
    print("Training time: {}".format(end-start))
    result_dict['history'] = history
    result_dict['intersedctions'] = isct_inc
    result_dict['train_time'] = end-start

    
    return result_dict, top_corr_df

In [37]:
def test_then_train_model(data, ae_score, number_of_cols, target, results_load_path, results_save_path, base_mode_path, pre_trained=True) :
    result_dict = {}

    # ------------------------------------ data processing ---------------------------------------- #
    top_corr_df = ae_score.corr()[target].sort_values(ascending=False)[:number_of_cols] # get the top correlated intersections
    isct_inc = top_corr_df.index.tolist()
    print("correlated intersections: {}".format(isct_inc))
    df = data[isct_inc].copy(deep=True)
    # df = df.mul(top_corr_df, axis=1)
    df = df.astype('float32')
    sequence_length = 12 # number of time steps to look back
    n_features = len(isct_inc) # number of features (correlated intersections)
    output_pred = 1 # number of time steps to predict
    n_obs = sequence_length * n_features # number of columns in the input
    reframed = series_to_supervised(df, sequence_length, output_pred)
    train_X, train_y, scl = preprocess_df(reframed, n_obs, n_features, sequence_length)


# # # ------------------------------------ modelling ---------------------------------------------- #
    # define model, loss function and optimizer
    model = LSTM_uni(input_dim = n_features, hidden_dim = 32, layer_dim = 1, output_dim = output_pred, dropout_prob= 0.2)
    model = model.to(device)
    loss_fn = torch.nn.MSELoss()
    optimiser = torch.optim.Adam(model.parameters(), lr=0.01)
    if pre_trained==True:
        model, optimiser = load_ckp(results_load_path, model, optimiser)
    

    # ------------------------------------ testing ---------------------------------------------- #
    yhat = model_evaluation( model, train_X , device)
    result_dict['RMSE'] = sqrt(mean_squared_error(yhat,train_y))
    result_dict['MAE'] = mean_absolute_error(yhat,train_y)
    result_dict['df'] = pd.DataFrame({"Real":train_y,"Predicted":yhat})
    print("RMSE: {}".format(result_dict['RMSE']))


   # ------------------------------------ training ---------------------------------------------- #
    start = time.time()
    if pre_trained==False:
        history, model = train_model(model, train_X,train_y, loss_fn, optimiser, device, epochs=50)
    else:
        history, model = train_model(model, train_X,train_y, loss_fn, optimiser, device, epochs=1)
    end = time.time()
    checkpoint = {'state_dict': model.state_dict(),'optimizer': optimiser.state_dict()}
    torch.save(checkpoint, results_save_path)
    print("Training time: {}".format(end-start))
    result_dict['history'] = history
    result_dict['intersedctions'] = isct_inc
    result_dict['train_time'] = end-start

    
    return result_dict, top_corr_df

In [38]:
def test_base_model(data, ae_score, number_of_cols, target, results_load_path, top_corr_df,pre_trained=True):
    result_dict = {}

    # ------------------------------------ data processing ---------------------------------------- #
    isct_inc = top_corr_df.index.tolist()
    print("correlated intersections: {}".format(isct_inc))
    df = data[isct_inc].copy(deep=True)
    df = df.mul(top_corr_df, axis=1)
    df = df.astype('float32')
    sequence_length = 12 # number of time steps to look back
    n_features = len(isct_inc) # number of features (correlated intersections)
    output_pred = 1 # number of time steps to predict
    n_obs = sequence_length * n_features # number of columns in the input
    reframed = series_to_supervised(df, sequence_length, output_pred)
    train_X, train_y, scl = preprocess_df(reframed, n_obs, n_features, sequence_length)


# # # ------------------------------------ modelling ---------------------------------------------- #
    # define model, loss function and optimizer
    model = LSTM_uni(input_dim = n_features, hidden_dim = 32, layer_dim = 1, output_dim = output_pred, dropout_prob= 0.2)
    model = model.to(device)
    loss_fn = torch.nn.MSELoss()
    optimiser = torch.optim.Adam(model.parameters(), lr=0.01)
    if pre_trained==True:
        model, optimiser = load_ckp(results_load_path, model, optimiser)
    

    # ------------------------------------ testing ---------------------------------------------- #
    yhat = model_evaluation( model, train_X , device)
    result_dict['RMSE'] = sqrt(mean_squared_error(yhat,train_y))
    result_dict['MAE'] = mean_absolute_error(yhat,train_y)
    result_dict['df'] = pd.DataFrame({"Real":train_y,"Predicted":yhat})
    result_dict['intersedctions'] = isct_inc
    print("RMSE: {}".format(result_dict['RMSE']))
    
    return result_dict, top_corr_df

In [45]:
# ------------------------------------ for metr-la dataset ---------------------------------------- #
target = '737529'
threshold = 0.05
time_window = 24 # in hours
errors={}
errors[target]={}
errors[target][threshold]={}
errors[target][threshold]['incremental_weighted_update']={}
errors[target][threshold]['incremental_update']={}
errors[target][threshold]['No_update']={}

dfs={}
intersection_arrays = []
print("\n \n Starting target: {}".format(target))
print("Starting threshold: {}".format(threshold))

# ------------------------------------ data processing for base model ---------------------------------------- #
data = merge_trejectory_data_metr(results)# get raw data of the current trajectory and direction
ae_score = correlated_results['df'] # AE scores of the current trajectory and direction
data_size = data.shape[0]
base_train_portion = 0.5
model_start_time  = data.index[0]
model_end_time = data.index[-1]
baseline_time = data.iloc[int(data_size*base_train_portion)].name
incremental_time = baseline_time
base_ae = ae_score[ae_score.index<=baseline_time]
stream_ae = ae_score[ae_score.index>baseline_time]
base_data = data[data.index<=baseline_time]
stream_data = data[data.index>baseline_time]
number_of_cols = math.ceil(len(ae_score.columns)*threshold) # number of outlier weighted intersections
if number_of_cols==0: # if threshold is 0, then use the target intersection only
    number_of_cols=1

# ------------------------------------ training base model ---------------------------------------- #
print(f"training from {model_start_time} to {baseline_time}")
exp_name = 'univariate_AE_incremental_update_results_base_model.pt'
results_save_path = os.path.join(base_result_path, exp_name)
results_save_weighted_path  = results_save_path
base_model_path = results_save_path
results_load_path = "None"
results_load_weighted_path = "None"
model_results, fixed_base_corr_df = test_then_train_model(base_data, base_ae, number_of_cols, target, results_load_path, results_save_path,base_model_path, pre_trained=False)
errors[target][threshold]['base_model'] = model_results

# # ------------------------------------ training incremental models ---------------------------------------- #

incremental_time = incremental_time + pd.DateOffset(hours=time_window)
while(incremental_time<model_end_time):
    print(f"\n\ntraining from {baseline_time} to {incremental_time}")
    incremental_ae = stream_ae[(stream_ae.index>baseline_time) & (stream_ae.index<=incremental_time)]
    incremental_data = stream_data[(stream_data.index>baseline_time) & (stream_data.index<=incremental_time)]
    exp_name = 'univariate_AE_incremental_update_results_incremental_model_{}.pt'.format(incremental_time)
    exp_name_weighted = 'univariate_AE_incremental_update_results_incremental_weighted_model_{}.pt'.format(incremental_time)
    results_load_weighted_path = results_save_weighted_path
    results_save_weighted_path = os.path.join(base_result_path, exp_name_weighted)
    results_load_path = results_save_path
    results_save_path = os.path.join(base_result_path, exp_name)

    # ------------------------------------ training incremental models ---------------------------------------- #
    print("training incremental weighted update model")
    model_results,top_corr_df = test_then_train_weighted_model(incremental_data, incremental_ae, number_of_cols, target, results_load_weighted_path, results_save_weighted_path, base_model_path, pre_trained=True)
    errors[target][threshold]['incremental_weighted_update'][incremental_time] = model_results

    print("training incremental update model")
    model_results,top_corr_df = test_then_train_model(incremental_data, incremental_ae, number_of_cols, target, results_load_path, results_save_path, base_model_path, pre_trained=True)
    errors[target][threshold]['incremental_update'][incremental_time] = model_results

    print("training No update model")
    model_results,top_corr_df = test_base_model(incremental_data, incremental_ae, number_of_cols, target, base_model_path, fixed_base_corr_df, pre_trained=True)
    errors[target][threshold]['No_update'][incremental_time] = model_results


    baseline_time = incremental_time
    incremental_time = incremental_time + pd.DateOffset(hours=time_window)



 
 Starting target: 737529
Starting threshold: 0.05
training from 2012-03-01 00:00:00 to 2012-04-29 12:00:00
correlated intersections: ['737529', '717821', '767366', '767573', '765273', '717608', '718066', '765265', '760024', '773012', '773975']
RMSE: 58.97197096396728
Training time: 37.498077154159546


training from 2012-04-29 12:00:00 to 2012-04-30 12:00:00
training incremental weighted update model
correlated intersections: ['737529', '718072', '716571', '765171', '760987', '767495', '716949', '717821', '772669', '773012', '764853']
RMSE: 17.4532911110045
Training time: 0.1997079849243164
training incremental update model
correlated intersections: ['737529', '718072', '716571', '765171', '760987', '767495', '716949', '717821', '772669', '773012', '764853']
RMSE: 17.4532911110045
Training time: 0.01681804656982422
training No update model
correlated intersections: ['737529', '717821', '767366', '767573', '765273', '717608', '718066', '765265', '760024', '773012', '773975']
RMSE: 17

In [46]:
# ------------------------------------ save results ---------------------------------------- #
with open(os.path.join(base_result_path, f'univariate_AE_real_time_processing_threshold_{threshold}_{time_window}H_results_{target}.pkl'), 'wb') as f:
    pickle.dump(errors, f)